# Progress reports
Generate a randomly sampled list of team members to do internal progress reports

In [1]:
# Import necessary modules
import requests
from bs4 import BeautifulSoup
import pandas

In [2]:
# Load and parse our team website
page = requests.get("http://www.anaweb.ch/ueber_uns/abteilungen/index_ger.php?key=klin").text
soup = BeautifulSoup(page, 'html.parser')

In [3]:
# Generate dataframe to save the data
Team = pandas.DataFrame()
# Get all names from our page, and save them to a string
Team['Name'] = [str(n.string) for n in soup.find_all(itemprop="name")]
# Drop 'Universität Bern', which we don't want in the list
Team = Team[Team.Name != 'Universität Bern']
# Get them functions as strings
Team['JobTitle'] = [str(i.string) for i in soup.find_all(itemprop="jobTitle")]
# In the first iteration, no one is presenting :)
Team['Presenting'] = False

In [4]:
# Based on https://stackoverflow.com/a/13842286/323100 we set some JobTitles to presenting
for index, row in Team.iterrows():
    # Set all 'Doktoranden' and 'Post-Docs' to True
    if 'Do' in row.JobTitle:
        Team.at[index, 'Presenting'] = True
    # Set all 'Assis' and 'Hilfs-Assis' to True   
    if 'ssi' in row.JobTitle:
        Team.at[index, 'Presenting'] = True
    # I guess Valentin should present, too
    if 'Geschäftsführender' in row.JobTitle:
        Team.at[index, 'Presenting'] = True

In [5]:
# These persons don't 'Show & Tell'
print('Of %s team members in total, these %s persons *do not* talk about their work in the labmeeting' %(len(Team),
                                                                                                         len(Team[Team['Presenting'] == False])))
Team[Team['Presenting'] == False]

Of 26 team members in total, these 9 persons *do not* talk about their work in the labmeeting


,Name,JobTitle,Presenting
2,Johannes Dominik Bastian,Gastarzt aus Orthopädischer Chirurgie und Trau...,False
3,Mathias Bergmann,None,False
4,Nane Boemke,Leiterin der Prosektur und Organisation Klinis...,False
5,Regula Buergy-Meyer,Laborantin,False
8,Pascal Graa,None,False
9,Hans Grossmann,Präparator,False
15,Adolfo Odriozola,Laborant,False
22,Jeannine Wagner-Zimmermann,Laborantin,False
23,Katrin Weber,None,False


In [6]:
# Put an empty presenting date in a new column
Team['Presented'] = False

In [7]:
# Based on https://stackoverflow.com/a/11531402/323100 we set the dates of the team members that already presented
Team.loc[Team.Name.str.contains('Cédric'), 'Presented'] = '2018-05-29'
Team.loc[Team.Name.str.contains('Anita'), 'Presented'] = '2018-07-03'
Team.loc[Team.Name.str.contains('Matthias'), 'Presented'] = '2018-09-18'
Team.loc[Team.Name.str.contains('Verdiana'), 'Presented'] = '2018-10-02'

In [8]:
# We already had progress reports of these persons
print('We already had (or have planned) a progress report')
for i, member in Team[Team['Presenting'] & Team['Presented']].sort_values(by='Presented').iterrows():
    print('- of', member['Name'], 'on', pandas.to_datetime(member['Presented']).strftime('%B %d in %Y'))

We already had (or have planned) a progress report
- of Cédric Zubler on May 29 in 2018
- of Anita Senk on July 03 in 2018
- of Matthias Messerli on September 18 in 2018
- of Verdiana Trappetti on October 02 in 2018


In [9]:
# These persons remain
print('These are the upcoming people in a random but preferred order.')
print('Volounteers will be treated preferentially.')
# Conventiently, 'random_state=11' samples the team member list in a way that Cedric is last and Anita second to last.
# Those two volounteered before I started this notebook...
# We thus just flip the dataframe for display and we're done.
for i, member in Team[Team['Presenting'] == True].sample(frac=1, random_state=11).iloc[::-1].iterrows():
    if not member['Presented']:
        print('\t-', member['Name'])

These are the upcoming people in a random but preferred order.
Volounteers will be treated preferentially.
	- Dea Aaldijk
	- Oleksiy-Zakhar Khoma
	- Sara Soltermann
	- Moritz von Werdt
	- Cristian Fernandez Palomo
	- Sebastian Halm
	- David Haberthür
	- Valentin Djonov
	- Helena Röss
	- Fluri Wieland
	- Ruslan Hlushchuk
	- Thomas Alexander Gorr
	- Marine Potez


Now send an calendar update to the group with the addresses from the (internal) [Multimail Tool](https://www.anaintra.unibe.ch/multimail.php).